<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>

### 3.2 Model Building with Traffic
---


<div style="text-align: justify; line-height: 2">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Having observed that weather conditions do not have a significant impact on the end delay of STM buses, we will attempt to build a model that also includes traffic data to see if it can improve the model's predictive ability. The new model will include the same weather and bus data as the previous unfiltered model. This data will be combined with te traffic data from the City of Montreal. [See master+traffic dataset description](../Data/Dataset_descriptions.md). A sample of the data is shown below.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|    |   line |   delay_start |   start_time |   temp_start |   rain_amt |   snow_amt |   delay_end |   total_traffic |   T_NBLT |   T_NBT |   T_NBRT |   T_SBLT |   T_SBT |   T_SBRT |   T_EBLT |   T_EBT |   T_EBRT |   T_WBLT |   T_WBT |   T_WBRT |
|---:|-------:|--------------:|-------------:|-------------:|-----------:|-----------:|------------:|----------------:|---------:|--------:|---------:|---------:|--------:|---------:|---------:|--------:|---------:|---------:|--------:|---------:|
|  0 |    467 |             0 |          405 |          7.1 |          0 |          0 |           6 |            3674 |        0 |    1236 |       23 |        4 |    1270 |      155 |      165 |     157 |       68 |        2 |     478 |      116 |
|  1 |    467 |             1 |          437 |          6.9 |          0 |          0 |           5 |            3116 |        0 |    1031 |       56 |        0 |     927 |      103 |      130 |     238 |       45 |        0 |     473 |      113 |
|  2 |    467 |             1 |          468 |          6.9 |          0 |          0 |           6 |            3984 |        0 |    1330 |       44 |        0 |    1167 |      170 |      191 |     312 |       89 |        3 |     567 |      111 |
|  3 |    467 |            -1 |          496 |          6.5 |          0 |          0 |           6 |            2655 |        0 |     853 |       35 |        0 |     884 |      113 |      116 |     171 |       86 |        1 |     321 |       75 |
|  4 |    467 |             0 |          527 |          6.5 |          0 |          0 |           6 |            2273 |        0 |     762 |       32 |        0 |     774 |       81 |       96 |     164 |       47 |        1 |     253 |       63 |





</div>
&nbsp;  
&nbsp;  

---

<a name="footnote1"></a>1: [PennState Eberly College of Science](./5_References.ipynb#1)  
<a name="footnote2"></a>2: [Ivan T. Ivanov](./5_References.ipynb#2)  
<a name="footnote3"></a>3: [OpenStax](./5_References.ipynb#3)  
<a name="footnote4"></a>4: [Heij, De Boer; Franses, Kloep; and Van Dijk](./5_References.ipynb#4)  
<a name="footnote5"></a>5: [Mostoufi, Navid; Constantinides, Alkis](./5_References.ipynb#5)

<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<span style="flex: 1; text-align: center;">3.2 Model Building with Traffic</span>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>
